In [1]:
import numpy as np
import pandas as pd
from dateutil import parser
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn  as sns
%matplotlib inline

In [2]:
test = pd.read_csv('Test-Set.csv')
train = pd.read_csv('Train-Set.csv')

In [3]:
test.head()

,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
train.head()

,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
# The workflow
workflow = ['1. Get data ready',
 '2. Pick a model(estimator) or algorithm to suit your problem',
 '3. Fit the model to the data and make a prediction',
 '4. Evaluate the model',
 '5. Improve the model',
 '6. Save and reload a trained model',
 '7. Putting it all together']  # topics

# **Getting the data ready, fixing NaNs**

In [6]:
# checking data types 
train.dtypes

ProductID             object
Weight               float64
FatContent            object
ProductVisibility    float64
ProductType           object
MRP                  float64
OutletID              object
EstablishmentYear      int64
OutletSize            object
LocationType          object
OutletType            object
OutletSales          float64
dtype: object

In [7]:
# investigating the training dataset 
train.isna().sum()

ProductID               0
Weight               1463
FatContent              0
ProductVisibility       0
ProductType             0
MRP                     0
OutletID                0
EstablishmentYear       0
OutletSize           2410
LocationType            0
OutletType              0
OutletSales             0
dtype: int64

In [29]:
test.isna().sum()

ProductID               0
Weight                976
FatContent              0
ProductVisibility       0
ProductType             0
MRP                     0
OutletID                0
EstablishmentYear       0
OutletSize           1606
LocationType            0
OutletType              0
dtype: int64

## **fill missing values**

In [8]:
# fill missing values for train dataset
train.Weight.fillna(train.Weight.mean(), inplace = True)
train.OutletSize.fillna ('missing', inplace = True)

In [30]:
# fill missing values for our test dataset
test.Weight.fillna(train.Weight.mean(), inplace = True)
test.OutletSize.fillna ('missing', inplace = True)

In [31]:
X = train.drop('OutletSales', axis = 1)
y = train['OutletSales']

In [32]:
X.shape , y.shape

((8523, 11), (8523,))

In [33]:
X.dtypes

ProductID             object
Weight               float64
FatContent            object
ProductVisibility    float64
ProductType           object
MRP                  float64
OutletID              object
EstablishmentYear      int64
OutletSize            object
LocationType          object
OutletType            object
dtype: object

## **Now convert the non numerical field / columns to numeric**

In [34]:
# changing the car categories into to numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#****************************************************************************************************

# now defime our categorical features
categorical_features  = ['ProductID','FatContent','ProductType','OutletID', 'OutletSize','LocationType','OutletType']
one_hot = OneHotEncoder()
transformer = ColumnTransformer ([("one_hot",
                                   one_hot,
                                   categorical_features)],
                                 remainder= 'passthrough')   # will take in a turple
                                 
#****************************************************************************************************

transformed_X = transformer.fit_transform(X)
transformed_X    # this is X that has undergone cleaning

<8523x1605 sparse matrix of type '<class 'numpy.float64'>'
	with 93227 stored elements in Compressed Sparse Row format>

## **Now selecting the right prediction model - we will use Forest Regressor**

In [35]:
# choosing the right prediction model

np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor

#****************************************************************************************************

# split and train model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(transformed_X, y, test_size =0.2)

##  transformed_X is  X that has undergone cleaning

#****************************************************************************************************
# fit the model
model = RandomForestRegressor()

model.fit(X_train,y_train)

model.score(X_test, y_test)

0.5613873724063994